<a href="https://colab.research.google.com/github/jport2/bime/blob/master/beginner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2019 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# MNIST Classification with 3 Activation Functions

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/quickstart/beginner"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/quickstart/beginner.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

In [0]:
import tensorflow as tf

Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [0]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


We will examine the problem with 3 different activation functions. First, we will examine the traditional relu we learned in class. Then we will use the sigmoid function which is popular for logistic regression. Lastly, we will examine the tanh activation function.

In [0]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])
model2 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='sigmoid'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])
model3 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='tanh'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [0]:
predictions = model(x_train[:1]).numpy()
predictions2 = model2(x_train[:1]).numpy()
predictions3 = model3(x_train[:1]).numpy()

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [0]:
tf.nn.softmax(predictions).numpy()
tf.nn.softmax(predictions2).numpy()
tf.nn.softmax(predictions3).numpy()

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

In [0]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_fn2 = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
loss_fn3 = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`.

In [0]:
loss_fn(y_train[:1], predictions).numpy()
loss_fn2(y_train[:1], predictions2).numpy()
loss_fn3(y_train[:1], predictions3).numpy()

In [0]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])
model2.compile(optimizer='adam',
              loss=loss_fn2,
              metrics=['accuracy'])
model3.compile(optimizer='adam',
              loss=loss_fn3,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [0]:
model.fit(x_train, y_train, epochs=5)
model2.fit(x_train, y_train, epochs=5)
model3.fit(x_train, y_train, epochs=5)

The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [0]:
model.evaluate(x_test,  y_test, verbose=2)
#relu

The image classifier is now trained to ~98% accuracy on this dataset with the relu activation function.

In [0]:
model2.evaluate(x_test,  y_test, verbose=2)
#sigmoid

The image classifier is now trained to ~97% accuracy on this dataset with the sigmoid function.

In [0]:
model3.evaluate(x_test,  y_test, verbose=2)
#tanh

The image classifier is now trained to ~97% accuracy on this dataset with the tanh function.

The image classifier achieves the highest prediction accuracy with the relu activation function which weighs all negative numbers as 0.